In [5]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 4.7.10
  latest version: 4.8.3

Please update conda by running

    $ conda update -n base -c defaults conda



## Package Plan ##

  environment location: /Users/attreys/anaconda3

  added / updated specs:
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    conda-4.8.3                |   py37hc8dfbb8_1         3.0 MB  conda-forge
    geographiclib-1.50         |             py_0          34 KB  conda-forge
    geopy-1.21.0               |             py_0          58 KB  conda-forge
    python_abi-3.7             |          1_cp37m           4 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         3.1 MB

The following NEW packages will be INSTALLED:

  geographiclib      conda-forge/noarch::geograp

In [4]:
!pip install geopy

     |████████████████████████████████| 112kB 107kB/s eta 0:00:01


In [449]:
import requests
import urllib.request
import time
from bs4 import BeautifulSoup
import numpy as np
import pandas as pd
from urllib.request import urlopen
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
html = urlopen(url) 
soup = BeautifulSoup(html, 'html.parser')

In [451]:
tables = soup.find_all('table')

In [452]:
type(tables)

bs4.element.ResultSet

In [454]:
req_table=tables[0]

In [455]:
for i in req_table:
    rows2=req_table.find_all('td')

In [456]:
len(rows2)

180

In [458]:
pincode=[]
burough=[]
neighbourhood=[]
for i in range(0,len(rows2)):
    if(len(rows2[i].findAll('a'))==1):
        pincode.append(rows2[i].findAll('b')[0].text)
        burough.append(rows2[i].findAll('a')[0].get('title'))
        #If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough.
        neighbourhood.append(rows2[i].findAll('a')[0].get('title'))
    elif(len(rows2[i].findAll('a'))>=2):
        for j in range(1,len(rows2[i].findAll('a'))):
            pincode.append(rows2[i].findAll('b')[0].text)
            burough.append(rows2[i].findAll('a')[0].get('title'))
            neighbourhood.append(rows2[i].findAll('a')[j].get('title'))
    else:
        pincode.append(rows2[i].findAll('b')[0].text)
        burough.append('0')
        neighbourhood.append('0')

In [459]:
type(pincode)

list

In [462]:
len(pincode)

252

In [463]:
len(burough)

252

In [465]:
len(neighbourhood)

252

In [466]:
import pandas as pd
df_final = pd.DataFrame(
    {'PostalCode': pincode,
     'Borough': burough,
     'Neighborhood': neighbourhood
    })


Only process the cells that have an assigned borough. Ignore cells with a borough that is Not assigned.

In [469]:
df_fin=df_final[df_final['Borough']!='0']

In [471]:
df2=df_fin.groupby("PostalCode").count()['Neighborhood']>1

In [472]:
df2=df2.reset_index()

In [474]:
list_pincode_single=df2[df2['Neighborhood']==False]['PostalCode'].tolist()

In [476]:
list_pincode_multiple=df2[df2['Neighborhood']==True]['PostalCode'].tolist()

In [492]:
cols={'PostalCode','Borough','Neighborhood'}
k=pd.DataFrame(columns=cols)
for i in range(0,len(df_fin)):
    if(df_fin.iloc[i]['PostalCode'] in list_pincode_single):
        k=k.append(df_fin.iloc[i])  

In [482]:
len(list_pincode_multiple)

43

More than one neighborhood can exist in one postal code area. For example, in the table on the Wikipedia page, you will notice that M5A is listed twice and has two neighborhoods: Harbourfront and Regent Park. These two rows will be combined into one row with the neighborhoods separated with a comma as shown in row 11 in the above table.

In [483]:
pincode2=[]
burough2=[]
neighbourhood2=[]

In [484]:
for j in range(0,len(list_pincode_multiple)):
        pincode2.append(list_pincode_multiple[j])
        #print(pincode2)
        burough2.append(df_fin[df_fin['PostalCode']==list_pincode_multiple[j]]['Borough'].iloc[0])
        str1=""
        neighbourhood2.append(str1.join(df_fin[df_fin['PostalCode']==list_pincode_multiple[j]]['Neighborhood'].tolist()))  

In [485]:
len(pincode2)

43

In [486]:
len(burough2)

43

In [487]:
len(neighbourhood2)

43

In [488]:
import pandas as pd
df_multipleeq = pd.DataFrame({'PostalCode': pincode2,'Borough': burough2,'Neighborhood': neighbourhood2})


In [496]:
len(k)

58

In [499]:
final=df_multipleeq.append(k)

In the last cell of your notebook, use the .shape method to print the number of rows of your dataframe.

In [501]:
final.shape

(101, 3)